# Анализ вакансий DS, DA в телеграм каналах

#  План работы над проектом

**Цель проекта**:
* Составить портрет сотрудника, которого чаще всего готовы принять на позиции DS, DA, а также оценить условия труда, предлогаемые в этих вакансиях. На основе анализа можно будет проследить как меняется рынок труда, выделить тенденции. Это поможет соискателям лучше понимать нанимающие компании, оценить и сопоставить свои ожидания и предложения компаний.

**При описании портрета сотрудника и оценке условий труда, будем учитывать следующее:**
* образование
* нанимающую компанию
* размер компании
* опыт сотрудника (грейд)
* требуемый стек технологий
* формат работы (удаленный/офис)
* зарплата

**В ходе исследования проверим следующие гипотезы:**
1. Больше всего вакансий, предлагающих работу в офисе, чем удаленный формат.
2. Приблизиттельное одинаковое количество вакансий для DA и DS
3. Больше вакансий от крупных компаний, чем от стартапов. 
4. Больше всего вакансий для специалистов с опытом, чем для junior специалистов.
5. В зимнее месяцы количество вакансий больше, чем в летние месяцы. 




# Этапы выполнения проекта.
1. Парсер телеграм-каналов с вакансиями DS, DA.
        * выбрать необходимые телеграм-каналы
        * выбрать период за который будут анализироваться вакансии
        * установить лимит выгружаемых вакансий
        * сформировать csv-файл с первоначальными данными о вакансиях
2. Предобработка данных:
        * Очистить и подготовить текст для дальнейшей работы (с использованием регулярных выражений, лемматизация, библиотеки    ntlk с стоп-словами).
        * Исследовать пропущенные значения
        * Проверить наличие дубликатов
        * Работа с типами данных
        * Создание необходимых колонок
3. Исследовательский анализ данных:
        * Обнаружение аномалий, выбросов, интересных фактов и зависимостей.
        * Поиск ответов на выдвинутые гипотезы
        * Визуальзация данных с 
4. Оформление результатов исследования в понятный и функциональный дашборд. 
        * визуализировать ответы на выдвинутые гипотезы
        * визуализировать потррет типичного специалиста DS и DA

Для анализа вакансий будем использовать следующие чаты.

* https://t.me/foranalysts
* https://t.me/analyst_job
* https://t.me/biheadhunter
* https://t.me/datasciencejobs
* https://t.me/bds_job
* https://t.me/datajobschannel
* https://t.me/data_hr

# Парсер телеграм-каналов

In [1]:
# !pip3 install pyrogram tgcrypto

In [2]:
# !pip3 install python-dotenv

In [3]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyrogram
import requests
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from pyrogram import Client
import pandas as pd
import json
from datetime import datetime
import re
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets
from urllib.parse import urlparse
import emoji

/var/folders/vf/z1gk_tyn2zsf73w6bspmxgjm0000gn/T/ipykernel_81140/844583093.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
/var/folders/vf/z1gk_tyn2zsf73w6bspmxgjm0000gn/T/ipykernel_81140/844583093.py:12: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [5]:
pyrogram.__version__

'2.0.51'

In [6]:
dotenv_path = os.path.join('dot.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [7]:
API_ID = os.environ['API_ID']
API_HASH = os.environ['API_HASH']

In [8]:
# выгружаем данные после работы скрипта
# %%bash
# python3 pyro_run.py

In [9]:
# читаем и сохраняем данные в переменную
telegram = pd.read_csv('telegram.csv')

In [10]:
telegram.head(3)

,chat,message_id,date,text,entities
0,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4730,2022-09-09 17:05:10,Product Analyst\nв Safeguard Global — поставщи...,"[\n {\n ""_"": ""MessageEntity"",\n ..."
1,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4729,2022-09-09 15:12:08,Бизнес-аналитик/Менеджер проекта\nв Simble — I...,"[\n {\n ""_"": ""MessageEntity"",\n ..."
2,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4728,2022-09-09 09:00:11,Lead Data Engineer\nв Cardsmobile — компания р...,"[\n {\n ""_"": ""MessageEntity"",\n ..."


In [11]:
telegram.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8046 entries, 0 to 8045
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chat        5048 non-null   object
 1   message_id  8046 non-null   int64 
 2   date        8046 non-null   object
 3   text        7512 non-null   object
 4   entities    7345 non-null   object
dtypes: int64(1), object(4)
memory usage: 314.4+ KB


In [12]:
# избавляемся от пропусков
telegram = telegram.dropna().reset_index(drop=True) 

In [13]:
telegram.shape

(4580, 5)

В отдельную колонку сохраним информацию о id чата и его названии.

In [14]:
telegram["chat_id"] = telegram["chat"].apply(lambda x: json.loads(x)['id'])
telegram["title_chat"] = telegram["chat"].apply(lambda x: json.loads(x)['title'])

In [15]:
# В колонке data изменит тип данных
telegram.date = pd.to_datetime(telegram['date'])

In [16]:
telegram.date.sort_values(ascending=False)

0      2022-09-09 17:05:10
1      2022-09-09 15:12:08
847    2022-09-09 14:39:26
2850   2022-09-09 13:46:35
1324   2022-09-09 10:02:44
               ...        
4320   2019-02-18 13:24:38
4321   2019-02-10 12:12:21
4322   2019-02-06 14:14:56
4323   2019-01-31 22:29:30
4324   2019-01-29 20:02:49
Name: date, Length: 4580, dtype: datetime64[ns]

In [17]:
telegram.head(1)

,chat,message_id,date,text,entities,chat_id,title_chat
0,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4730,2022-09-09 17:05:10,Product Analyst\nв Safeguard Global — поставщи...,"[\n {\n ""_"": ""MessageEntity"",\n ...",-1001137236002,Job for Analysts & Data Scientists


Кнопка для просмотра выгруженных вакансий.

In [18]:
button = widgets.Button(description="Показать")
output = widgets.Output()

display(button, output)

counter = 0
def on_clicked(b):
    with output:
        try:
            global counter
            clear_output()
            print('№ вакансии: ', counter)
            print(telegram['text'][counter])
            counter += 1
        except Exception as e:
            print("Вакансии закончились")
            
button.on_click(on_clicked)

Button(description='Показать', style=ButtonStyle())

Output()

## Вывод
* Данные о вакансиях парсили из 7 телеграм-каналов
* Всего 4580 строк
* Данные выгруженны за период с 29.01.2019 по 09.09.2022

# Извлечение информации по ссылкам в вакансиях

## Выгрузим ссылки в отдельную колонку датасета - `url`

In [19]:
# def get_links(x, regexp='http\S+'): ###
#     try:
#         for i in json.loads(x['entities']):
#             if i['type'] == 'MessageEntityType.TEXT_LINK':
#                 return i['url']
#             elif i['type'] == 'MessageEntityType.URL':
#                 url = re.findall(regexp, x['text'])[0]
#                 return url
#     except:
#         return None

In [20]:
def get_links(x, regexp='http\S+'): ###
    try:
        for i in json.loads(x['entities']):
            if i['type'] == 'MessageEntityType.TEXT_LINK':
                return i['url']
            elif i['type'] == 'MessageEntityType.URL':
                url = re.findall(regexp, x['text'])[0]
                return url
    except:
        return None

In [21]:
telegram['url'] = telegram.apply(get_links, axis=1)

In [22]:
telegram['url'][0]

'https://www.linkedin.com/jobs/view/3243717640/'

## Найдем наиболее встречающиеся сайты, с которых поступает информация о вакансиях

In [23]:
# функция для поиска hostname url ссылки
def get_hostname(link):
    o = urlparse(link)
    host = o.hostname
    return host

In [24]:
telegram['host'] = telegram['url'].apply(lambda x: get_hostname(x))

Выведем 30 hostname, ссылка на которые встречаются больше всего. Будем вытаскивать информацию из ссылки, если она встречается больше 5 раз и ведет на сайт с вакансиями. 

In [25]:
telegram.groupby('host')['text'].agg('count').sort_values(ascending=False).head(30)

host
geekjob.ru                                   733
ya.cc                                        203
hh.ru                                        131
t.me                                         115
purninja-st.s3.eu-central-1.amazonaws.com     40
is.gd                                         31
otus.ru                                       26
i.ibb.co                                      26
spb.hh.ru                                     24
practicum.yandex.ru                           23
www.notion.so                                 23
telegra.ph                                    14
proglib.io                                    14
clck.ru                                       14
www.youtube.com                               12
ctrl2go.solutions                             12
praktikum.yandex.ru                           12
www.linkedin.com                              10
netology.ru                                    9
yandex.ru                                      9
your.gms.tech  

## Напишем функции для парсинга ссылок

In [26]:
class RequestVacancy:
    USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    
    def get_vacancy(self, url):
        if not url:
            return
        
        try:
            return requests.get(
                url, 
                headers=self._get_headers(),
#                 verify=False,
                timeout=20,
            )
        except ConnectionError:
            print(f'ConnectionError:\t{url}')
        except TimeoutError:
            print(f'TimeoutError:\t{url}')
        except InvalidSchema:
            print(f'InvalidSchema:\t{url}')
        except Exception as e:
            print(f'{type(e)}:\t{url}')
    
    def _get_headers(self) -> dict:
        return {
            'User-Agent': self.USER_AGENT,
        }

# vacancy requester
vc = RequestVacancy()

In [27]:
# def parse_site(url, element='div', extra_info=None):
#     try:
#         data = requests.get(url).text 
#         soup = BeautifulSoup(data)
#         elements = []
#         for extra in extra_info:
#             elements += soup.find_all(element, extra)
#         html = ''
#         for i in elements:
#             html += str(i)
#         return html
#     except Exception as e:
#         return '<p>Не удалось получить элемент</p>'

In [28]:
# HTML(parse_site(telegram['url'][1], element='section', extra_info=[{'class': 'col s12 m12 main'}]))

Функции для парсинга ссылок

In [29]:
from requests.exceptions import InvalidSchema

def default_parser(url):
    title, company, location, grade, salary, description = None, None, None, None, None, None
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }


def _get_domain(url):
    """
    'https://www.linkedin.com/jobs/view/2865172210/' => 'www.linkedin.com'
    """
    return '.'.join(urlparse(url).netloc.split('.')[-2:])


def get_text_from_url(url):
    response = vc.get_vacancy(url)
    if not response:
        return
    if response.status_code > 305:
        print(f'Status code: {response.status_code}')
        return
    return response.text



def apply_info(url):
    text = get_text_from_url(url)
    if not text:
        return
    
    parser = parsers.get(_get_domain(url))
    if not parser:
        parser = parsers['default']
    
    final_text = None
#     final_text = parser(text)
#     return json.dumps(final_text)
    try:
        final_text = parser(text)
    except Exception as e:
        print(f'{type(e)}:\t{e}:\t{url}')
    else:
        return json.dumps(final_text)

In [30]:
# для сайта geekjob
def parse_geekjob(text):
    soup = BeautifulSoup(text) 
    
    title = soup.find('h1').text.strip()
    company = soup.select_one('a#company-web').text.strip()
    location = soup.find('div', {'class':"location"}).text.strip() 
    grade = soup.find('div', {'class': "category"}).text.strip()
    salary = soup.find('span', {'class': "salary"}).text.strip()
    description = soup.find('div', {'class': "description"}).text.strip()
        
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }

In [31]:
# для сайта hh.ru
def parse_hh(text):
    
    soup = BeautifulSoup(text)

    title = soup.find('h1', {'data-qa':"vacancy-title"}).text.strip() #*
    company = soup.find('span', {'class': "vacancy-company-name"}, limit=1).text.strip()
    location = soup.find('p', {'data-qa':"vacancy-view-location"}).text.strip()
    grade = soup.find('p', {'class': "vacancy-description-list-item"}, limit=1).text.strip()
    salary = soup.find('div', {'data-qa': "vacancy-salary"}, limit=1).text.strip() #*
    description = soup.find('div', {'class': "vacancy-section"}).text.strip()
#    title = re.sub(r'[^а-яА-Яa-zA-Z]', ' ', title)
    
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }

In [32]:
# для сайта proglib
def parse_proglib(text):
    
    soup = BeautifulSoup(text)
    
    title = soup.find('h1', {'class':"entity__title mb-2"}).text.strip() #*
    company = soup.find('span', {'itemprop': "hiringOrganization"}).text.strip()
    location = soup.find('p', {'data-qa':"vacancy-view-location"}).text.strip()
    grade = soup.find('span', {'itemprop': "experienceRequirements"}).text.strip()
    salary = soup.find('span', {'itemprop': "baseSalary"}).text.strip() #*
    description = soup.find('div', {'class': "block__content entity__content"}).text.strip()
#    title = re.sub(r'[^а-яА-Яa-zA-Z]', ' ', title)
    
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }

In [33]:
# для сайта linkedn
def parse_linkedn(text):
    
    soup = BeautifulSoup(text)
    
    title = soup.find('h1', {'class':"top-card-layout__title"}).text.strip() #*
    company = soup.find('a', {'class': "topcard__org-name-link"}).text.strip()
    location = soup.find('span', {'class':"topcard__flavor topcard__flavor--bullet"}).text.strip()
    grade = soup.find('h3', {'class': "description__job-criteria-subheader"}).text.strip()
    salary = 'None' #*
    description = soup.find('div', {'class': "show-more-less-html__markup"}).text.strip()
#    title = re.sub(r'[^а-яА-Яa-zA-Z]', ' ', title)
    
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }

In [34]:
# для сайта yandex
def parse_yandex(text):
    
    soup = BeautifulSoup(text)

    title = soup.find('h1', {'class':"lc-styled-text__text"}).text.strip() #*
    company = 'Yandex'
    #     location = soup.find('span', {'class':"Text Text_typography_control-s lc-jobs-tag__label"}).text.strip()
    #     grade = soup.find('div', {'class': "lc-jobs-vacancy__meta-section-items"}).text.strip()

    location_tag = soup.select_one('div.lc-jobs-vacancy__meta-section:nth-child(3)')
    location_items = location_tag.select('span.lc-jobs-tag__label')
    location = ', '.join((t.text for t in location_items))

    grade_items = (
            soup
            .select_one('div.lc-jobs-vacancy__meta-section:nth-child(5)')
            .select('span.lc-jobs-tag__label')
        )
    grade = ', '.join((t.text for t in grade_items))


    salary = 'None' #*
    description = soup.find('div', {'class': "lc-jobs-vacancy__description"}).text.strip()
 
    #title = re.sub(r'[^а-яА-Яa-zA-Z]', ' ', title)
    
    return {
        "title": title,
        "comapany": company,
        "location": location,
        "grade": grade,
        "salary": salary,
        "description": description
    }

In [35]:
parsers = {
    'geekjob.ru': parse_geekjob,
    'ya.cc' : parse_yandex,
    'linkedin.com': parse_linkedn,
    'hh.ry': parse_hh,
    'default': default_parser
}

In [36]:
# parse_yandex(get_text_from_url('https://yandex.ru/jobs/vacancies/%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA-%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C-%D1%81%D0%B8%D0%BC%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D0%B8-%D0%B0%D0%B3%D0%B5%D0%BD%D1%82%D0%BE%D0%B2-%D0%B2-%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81-%D0%B1%D0%B5%D1%81%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D1%85-%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%BE%D0%B1%D0%B8%D0%BB%D0%B5%D0%B9-5773'))

In [37]:
# parse_linkedn(get_text_from_url('https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3253779996'))

In [38]:
from tqdm import tqdm
tqdm.pandas()
#df_users.groupby(['userID', 'requestDate']).progress_apply(feature_rollup)

In [39]:
telegram['info'] = telegram['url'].progress_apply(lambda x: apply_info(x))

  0%|          | 14/4580 [00:36<2:54:15,  2.29s/it]

<class 'AttributeError'>:	'NoneType' object has no attribute 'text':	https://www.linkedin.com/jobs/view/3247021542/


  0%|          | 15/4580 [00:37<2:12:09,  1.74s/it]

<class 'AttributeError'>:	'NoneType' object has no attribute 'text':	https://geekjob.ru/sfQ


  2%|▏         | 70/4580 [03:55<4:01:04,  3.21s/it]

...

 99%|█████████▊| 4514/4580 [1:08:06<00:31,  2.09it/s]

<class 'requests.exceptions.ConnectionError'>:	https://www.ngrow.ai)


 99%|█████████▉| 4540/4580 [1:08:13<00:12,  3.27it/s]

<class 'requests.exceptions.ConnectionError'>:	https://anyquery.diginetica.com)


100%|██████████| 4580/4580 [1:08:31<00:00,  1.11it/s]


In [64]:
telegram['info'][60]

'{"title": "\\u0421\\u0438\\u0441\\u0442\\u0435\\u043c\\u043d\\u044b\\u0439 \\u0430\\u043d\\u0430\\u043b\\u0438\\u0442\\u0438\\u043a 1\\u0421 ERP", "comapany": "icl-services.com", "location": "\\u041a\\u0430\\u0437\\u0430\\u043d\\u044c, \\u0420\\u043e\\u0441\\u0441\\u0438\\u044f", "grade": "\\u041c\\u0438\\u0434\\u0434\\u043b \\u2022 \\u0421\\u0435\\u043d\\u044c\\u043e\\u0440 \\u2022 \\u041a\\u043e\\u043d\\u0441\\u0443\\u043b\\u044c\\u0442\\u0430\\u043d\\u0442", "salary": "", "description": "\\u041e\\u043f\\u0438\\u0441\\u0430\\u043d\\u0438\\u0435 \\u0432\\u0430\\u043a\\u0430\\u043d\\u0441\\u0438\\u0438\\u0414\\u043b\\u044f \\u0431\\u043b\\u043e\\u043a\\u0430 \\"\\u0411\\u044e\\u0434\\u0436\\u0435\\u0442\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u0438\\u0435\\" welcome-\\u0431\\u043e\\u043d\\u0443\\u0441 \\u0434\\u043e 100 000 gross. \\u0418 \\u0442\\u0430\\u043a\\u0436\\u0435 \\u0437\\u0430 \\u0443\\u0441\\u043f\\u0435\\u0448\\u043d\\u0443\\u044e \\u0440\\u0435\\u043a\\u043e\\u043c\\

In [66]:
telegram.to_csv('telegram_plus_link.csv')

**Очистка текста**

**Избавимся от смайликов и не нужных символов в вакансии**

С помощью регулярных выражений очистим описание вакансий, оставив только английские, русские буквы, цифры.

In [40]:
def clin_rex(text):
    text = re.sub(r'[^а-яА-Яa-zA-Z0-9]', ' ', text)
    text = re.sub(r'[^\w\d\s]', ' ', text)
    return text

In [41]:
telegram['text'] = telegram['text'].apply(lambda x: clin_rex(x))

In [42]:
telegram['text'][0]

'Product Analyst в Safeguard Global   поставщик аутсорсинговых услуг по расчету заработной платы и управлению персоналом  Удаленная работа  Описание вакансии'

In [43]:
LABEL_1 = vac_name
LABEL_2 = type_of_work
LABEL_3 = company
LABEL_4 = stack


TRAIN_DATA = [
    (
        "Product Analyst в Safeguard Global поставщик аутсорсинговых услуг по расчету заработной платы и управлению персоналом Удаленная работа",
        {"entities": [(45, 54, LABEL)]},
    ),
    ("Анализ тестовых данных и данных в системах", {"entities": []}),
    (
        "Можно удаленно",
        {"entities": [(6, 14, LABEL)]},
    ),
    ("Удаленный формат работы", {"entities": [(0, 9, LABEL)]}),
    
]

NameError: name 'vac_name' is not defined